### Все импорты

In [ ]:
%%capture
%pip install transformers datasets accelerate rouge-score nltk bitsandbytes peft

In [ ]:
import json
import math
from tqdm import tqdm
import os
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from datasets import load_dataset

### Загрузка данных для обучения

### Код для загрузки различных моделей (можете использовать любые, но не берите слишком тяжеловесные, на бесплатных гпу с колаба вы не успеете ее обучить :)))

### Класс для обучения

In [ ]:
class ModelTrainer:
    def __init__(self, model_name: str, task: str = "causal", quantization: bool = True):
        self.model_name = model_name
        self.task = task
        self.quantization = quantization
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.tokenizer = self.load_model()

    def load_model(self):
        kwargs = {"load_in_4bit": True, "device_map": "auto"} if self.quantization else {"device_map": "auto"}

        if self.task == "causal":
            model = AutoModelForCausalLM.from_pretrained(self.model_name, **kwargs)
        elif self.task == "classification":
            model = AutoModelForSequenceClassification.from_pretrained(self.model_name, **kwargs)
        else:
            raise ValueError("Неподдерживаемая задача")

        tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        model.to(self.device)
        return model, tokenizer

    def load_data(self, data_path: str):
        dataset = load_dataset("json", data_files=data_path)
        return dataset.map(lambda examples: self.tokenizer(examples["question"] + " " + examples.get("context", ""),
                                                      truncation=True, padding="max_length", max_length=512),
                                                      batched=True)

    def train(self, train_data_path: str, output_dir: str = "./results", epochs: int = 3, batch_size: int = 2):
        os.makedirs(output_dir, exist_ok=True)

        tokenized_datasets = self.load_data(train_data_path)
        train_dataloader = torch.utils.data.DataLoader(tokenized_datasets["train"], batch_size=batch_size, shuffle=True)

        self.model.train()
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=5e-5)

        for epoch in range(epochs):
            progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)

            for batch in progress_bar:
                batch = {k: v.to(self.model.device) for k, v in batch.items()}
                optimizer.zero_grad()
                outputs = self.model(**batch)
                loss = outputs.loss
                loss.backward()
                optimizer.step()

                progress_bar.set_postfix(loss=loss.item())

        self.model.save_pretrained(output_dir)
        print(f"Модель {self.model_name} сохранена в {output_dir}.")

### Обучаем

In [ ]:
available_models = {
    "causal": ["ai-forever/saiga_mistral_7b", "mistralai/Mistral-7B-Instruct", "SberbankAI/FRED-T5-1.7B"],
    "classification": ["distilbert-base-uncased", "bert-tiny"]
} #гуглите любые, это просто примерус

selected_model = "ai-forever/saiga_mistral_7b"
trainer = ModelTrainer(selected_model)
trainer.train("train_data.json")

### Код для подсчета метрик вашей модельки :)

In [ ]:
from metrics import Validator #metrics.py

validator = Validator()

res = validator.validate_dataset(df, use_ragas=True) #df: пандас датасет с нужными полями: answer, ground_truth, context, question

# contex_rec = res["context_recall"]
# contex_prec = res["context_precision"]
# answer_cor_lit = res["answer_correctness_literal"]
# answer_cor_neu = res["answer_correctness_neural"]

### Графички, шоб красиво :)

In [ ]:
def plot_metric_distribution(results: dict):
    """Построить распределение каждой метрики."""
    plt.figure(figsize=(18, 12))
    for i, (metric, values) in enumerate(results.items(), 1):
        plt.subplot(2, 2, i)
        sns.histplot(values, bins=20, kde=True, color="skyblue")
        plt.title(f"Распределение {metric}")
        plt.xlabel("Оценка")
        plt.ylabel("Частота")
    plt.tight_layout()
    plt.show()

def plot_average_scores(results: dict):
    """Построить среднее значение для каждой метрики."""
    avg_scores = {metric: sum(values) / len(values) if values else 0.0 for metric, values in results.items()}

    plt.figure(figsize=(12, 8))
    sns.barplot(x=list(avg_scores.keys()), y=list(avg_scores.values()), palette="viridis")
    plt.title("Средние значения для каждой метрики")
    plt.xlabel("Метрика")
    plt.ylabel("Средняя оценка")
    plt.xticks(rotation=45)
    plt.show()

def plot_metrics_comparison(results: dict):
    """Построить сравнение метрик по образцам."""
    df = pd.DataFrame(results)
    plt.figure(figsize=(14, 10))
    sns.boxplot(data=df, palette="Set2")
    plt.title("Сравнение метрик (ящиковая диаграмма)")
    plt.xlabel("Метрика")
    plt.ylabel("Оценка")
    plt.xticks(rotation=45)
    plt.show()

def plot_correlation_heatmap(results: dict):
    """Построить тепловую карту корреляции между метриками."""
    df = pd.DataFrame(results)
    correlation = df.corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Тепловая карта корреляции метрик")
    plt.show()